# **SECOM Data Analysis and Model Training**

## **0. Data Download and Extraction**

The following cell downloads and extracts the data in case it is not already present in the local directory.

In [1]:
from get_data import download_and_extract_data

url = "https://archive.ics.uci.edu/static/public/179/secom.zip"
download_and_extract_data(url)

2026-01-19 21:29:19,122 - get_data - INFO - SECOM data found.
2026-01-19 21:29:19,123 - get_data - INFO - Data ready to be used.


## **1. Exploratory Data Analysis**

### **1.1 Analysis of Sensor Data**

#### **Load data**

In [2]:
import pandas as pd

In [3]:
data_path = r"data\secom.data"
df = pd.read_csv(data_path, sep=" ", header=None)

#### **Overview of data**

In [4]:
df.shape

(1567, 590)

In [5]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 590 entries, 0 to 589
dtypes: float64(590)
memory usage: 7.1 MB


All columns contain data of type float64, which means there are no categorical variables in the dataset.

#### **Check for constant features**

In [7]:
constant_features_mask = df.nunique(dropna=True) <= 1
print(f"{sum(constant_features_mask)} columns contain constant values. They will be removed in the preprocessing step.")
constant_features = df.columns[constant_features_mask]
constant_features.to_numpy()

116 columns contain constant values. They will be removed in the preprocessing step.


array([  5,  13,  42,  49,  52,  69,  97, 141, 149, 178, 179, 186, 189,
       190, 191, 192, 193, 194, 226, 229, 230, 231, 232, 233, 234, 235,
       236, 237, 240, 241, 242, 243, 256, 257, 258, 259, 260, 261, 262,
       263, 264, 265, 266, 276, 284, 313, 314, 315, 322, 325, 326, 327,
       328, 329, 330, 364, 369, 370, 371, 372, 373, 374, 375, 378, 379,
       380, 381, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       414, 422, 449, 450, 451, 458, 461, 462, 463, 464, 465, 466, 481,
       498, 501, 502, 503, 504, 505, 506, 507, 508, 509, 512, 513, 514,
       515, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538])

#### **Analysis of missing values by column**

In [8]:
print("Statistics on numbers of missing values per column:")
col_na_count = df.isna().sum(axis=0)
col_na_count.describe()

Statistics on numbers of missing values per column:


count     590.000000
mean       71.103390
std       241.850565
min         0.000000
25%         2.000000
50%         6.000000
75%         9.000000
max      1429.000000
dtype: float64

In [9]:
print(f"On average, each column has about {col_na_count.mean():.0f} missing values, while the highest number of missing values is {col_na_count.max()} (out of {len(df)} records).")

On average, each column has about 71 missing values, while the highest number of missing values is 1429 (out of 1567 records).


In [10]:
top_na_cols = 30
print(f"{top_na_cols} columns with most missing values:")
top_na_cols_df = df.isna().sum().sort_values(ascending=False)[:top_na_cols].to_frame().reset_index()
top_na_cols_df.columns = ["Col number", "Num NAs"]
top_na_cols_df

30 columns with most missing values:


,Col number,Num NAs
0,292,1429
1,293,1429
2,158,1429
3,157,1429
4,492,1341
5,85,1341
6,358,1341
7,220,1341
8,244,1018
9,517,1018


In [11]:
na_filter_threshold = 0.5
na_filter_mask = df.isna().sum()/len(df) > na_filter_threshold
print(
f"{sum(na_filter_mask)} columns have more than {na_filter_threshold:.0%} missing values.\n\
A high number of missing values could indicate an issue with the corresponding sensor. As \n\
no information regarding each feature is available, the arbitrary threshold of {na_filter_threshold:.0%} is used \n\
for deciding whether to drop a column. The columns that have a fraction of missing values \n\
that exceeds this threshold will be dropped in the preprocessing step."
)

28 columns have more than 50% missing values.
A high number of missing values could indicate an issue with the corresponding sensor. As 
no information regarding each feature is available, the arbitrary threshold of 50% is used 
for deciding whether to drop a column. The columns that have a fraction of missing values 
that exceeds this threshold will be dropped in the preprocessing step.


#### **Analysis and handling of missing values by row**

In [12]:
print("Statistics on numbers of missing values per row:")
df.isna().sum(axis=1).describe()

Statistics on numbers of missing values per row:


count    1567.000000
mean       26.771538
std        13.377518
min         4.000000
25%        20.000000
50%        24.000000
75%        32.000000
max       152.000000
dtype: float64

In [13]:
top_na_rows = 20
print(f"{top_na_rows} rows with most missing values:")
top_na_rows_df = df.isna().sum(axis=1).sort_values(ascending=False)[:top_na_rows].to_frame().reset_index()
top_na_rows_df.columns = ["Row number", "Num NAs"]
top_na_rows_df

20 rows with most missing values:


,Row number,Num NAs
0,1566,152
1,1564,148
2,1561,140
3,511,100
4,1152,100
5,810,99
6,93,96
7,95,96
8,814,96
9,995,92


In [14]:
max_na_count = (df.isna().sum(axis=1)).max()
max_na_fraction = max_na_count/df.shape[1]
print(f"The rows with the most missing values have at most {max_na_fraction:.0%} missing values.\nNo rows will be removed due to missing values.")

The rows with the most missing values have at most 26% missing values.
No rows will be removed due to missing values.


### **1.2 Analysis of Labels**

In [15]:
labels_path = r"data\secom_labels.data"
labels_df = pd.read_csv(labels_path, sep=" ", header=None)

In [16]:
labels_df.shape

(1567, 2)

In [17]:
labels_df.head(5)

,0,1
0,-1,19/07/2008 11:55:00
1,-1,19/07/2008 12:32:00
2,1,19/07/2008 13:17:00
3,-1,19/07/2008 14:43:00
4,-1,19/07/2008 15:22:00


In [18]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1567 non-null   int64 
 1   1       1567 non-null   object
dtypes: int64(1), object(1)
memory usage: 24.6+ KB


The labels contain no missing values.

In [19]:
labels_df[0].value_counts()

0
-1    1463
 1     104
Name: count, dtype: int64

In [20]:
labels_df[0].value_counts(normalize=True)

0
-1    0.933631
 1    0.066369
Name: proportion, dtype: float64

As mentioned in the dataset description, there are 104 test fails (which amount to 6.6% of tests), while all the other tests passed.

## **2. Preprocessing**

The data needs some preprocessing. The preprocessing steps are implemented below as steps that can be included in a Pipeline.

In [21]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

### **2.1 Remove Features With Many Missing Values**

In [22]:
class HighNaNColumnDropper(BaseEstimator, TransformerMixin):
    """Drop columns with fraction of NaNs greater than a threshold."""

    def __init__(self, nan_threshold=0.5):
        self.nan_threshold = nan_threshold
        self.keep_mask_ = None

    def fit(self, X, y=None):
        X = np.asarray(X)
        n_samples = X.shape[0]
        nan_frac = np.isnan(X).sum(axis=0) / float(n_samples)
        self.keep_mask_ = nan_frac <= self.nan_threshold
        return self

    def transform(self, X):
        X = np.asarray(X)
        return X[:, self.keep_mask_]

### **2.2 Remove Features With Constant Values**

In [23]:
class ConstantColumnDropper(BaseEstimator, TransformerMixin):
    """Drop columns that are constant (<= 1 unique non-NaN value)."""

    def __init__(self):
        self.keep_mask_ = None

    def fit(self, X, y=None):
        X = np.asarray(X)
        unique_counts = [len(np.unique(col[~np.isnan(col)])) for col in X.T]
        self.keep_mask_ = np.array([uc > 1 for uc in unique_counts], dtype=bool)
        return self

    def transform(self, X):
        X = np.asarray(X)
        return X[:, self.keep_mask_]

### **2.3 Impute Missing Values**
The median is used for imputing the missing values in each column. The Imputer from scikit-learn is used as is.

### **2.4 Correlation Analysis**

The correlation between the features will be computed, with the aim of dropping features that are highly correlated. This will help reduce the number of features. For each pair of highly correlated features, the one that has the lowest variance will be dropped.

In [24]:
class CorrelationFilter(BaseEstimator, TransformerMixin):
    """Remove highly correlated features (keep higher-variance feature)."""

    def __init__(self, corr_threshold=0.9):
        self.corr_threshold = corr_threshold
        self.keep_mask_ = None

    def fit(self, X, y=None):
        X = np.asarray(X)
        # Expect no NaNs (imputer should run before this)
        corr = np.corrcoef(X, rowvar=False)
        corr = np.abs(corr)
        n_features = corr.shape[0]
        to_drop = set()
        for i in range(n_features):
            if i in to_drop:
                continue
            for j in range(i + 1, n_features):
                if j in to_drop:
                    continue
                if corr[i, j] >= self.corr_threshold:
                    var_i = np.nanvar(X[:, i])
                    var_j = np.nanvar(X[:, j])
                    if var_i >= var_j:
                        to_drop.add(j)
                    else:
                        to_drop.add(i)
        keep = [i for i in range(n_features) if i not in to_drop]
        mask = np.zeros(n_features, dtype=bool)
        mask[keep] = True
        self.keep_mask_ = mask
        return self

    def transform(self, X):
        X = np.asarray(X)
        return X[:, self.keep_mask_]

### **2.5 Create Preprocessing Pipeline**

In [25]:
preproc_steps = Pipeline(
    [
        ("drop_nans", HighNaNColumnDropper(nan_threshold=0.5)),
        ("drop_constant", ConstantColumnDropper()),
        ("imputer", SimpleImputer(strategy="median")),
        ("corr_filter", CorrelationFilter(corr_threshold=0.9)),
    ]
)

### **2.6 Convert Data and Labels to Arrays**

In [26]:
X = df.to_numpy()

y = labels_df[0].to_numpy()
y = (y == 1).astype(int)  # Set -1 values to 0

## **3. Model Training**

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import (
    average_precision_score,
    f1_score,
    roc_auc_score,
    precision_recall_curve
)
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

### **3.1 Split Dataset**

In [28]:
test_size = 0.2
X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=test_size/(1-test_size), stratify=y_full_train, random_state=42)

In [29]:
len(X_train) + len(X_val) + len(X_test) == len(X)

True

In [30]:
len(y_train) + len(y_val) + len(y_test) == len(y)

True

### **3.2 Define Generic Training and Evaluation Functions**
To facilitate the fitting of multiple models, the 3 functions below are defined.

A grid search is performed to find the best combination of parameter values for each model. Since the number of measurements is relatively low for the number of features, cross-validation is implemented. Scoring is performed using the ROC-AUC and PR-AUC (average precision). The PR-AUC is used for selecting the best set of parameters for each model.

As the target labels are imbalanced (there are much fewer positive labels than negative), a search for the optimal threshold value is performed. The optimal threshold is found by using the best set of parameter values obtained from the grid search on the training data, and then computing the F1-score on the validation data.

The results of the grid search performed for each model, as well as the  are summarized in one dataframe.

In [31]:
def run_grid_search(X_train, y_train, pipeline, param_grid, cv_n_split=5):
    """Runs grid search with cross-validation for the provided parameter values.
    
    Returns grid search object.
    """
    cv = StratifiedKFold(n_splits=cv_n_split, shuffle=True, random_state=42)
    scoring = {"roc_auc": "roc_auc", "pr_auc": "average_precision"}

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        refit="pr_auc",  # Select the best model based on PR-AUC
        n_jobs=-1,
        verbose=2,
    )

    grid_search.fit(X_train, y_train)
    return grid_search

In [32]:
def select_threshold(grid_search, X_val, y_val):
    """Finds the probability threshold maximizing F1-score on validation data.
    
    Returns threshold and predicted labels.
    """
    estimator = grid_search.best_estimator_

    # Linear classifiers or trees may not have predict_proba (Ridge)
    if hasattr(estimator.named_steps["classifier"], "predict_proba"):
        probs = estimator.predict_proba(X_val)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_val, probs)
        if len(thresholds) == 0:
            best_threshold = 0.5
        else:
            p = precision[:-1]
            r = recall[:-1]
            denom = p + r
            f1_scores = np.divide(
                2 * (p * r), denom, out=np.zeros_like(denom), where=denom != 0
            )
            best_threshold = thresholds[np.argmax(f1_scores)]

    else:
        # Use decision_function for RidgeClassifier
        probs = estimator.decision_function(X_val)
        
        # F1-score is not meaningful for RidgeClassifier without calibration
        best_threshold = np.nan

    
    return best_threshold, probs

In [33]:
def summarize_results(model_name, grid_search, best_threshold, y_val, y_pred_proba):
    """Extracts the best CV results and calculates scores based on validation data.

    Returns dictionary with all metrics.
    """
    results = pd.DataFrame(grid_search.cv_results_)

    # Select the best model according to PR-AUC (refit metric)
    best_row = results.loc[results["rank_test_pr_auc"] == 1].iloc[0]

    # Linear classifiers or trees may not have predict_proba (Ridge)
    estimator = grid_search.best_estimator_
    if hasattr(estimator.named_steps["classifier"], "predict_proba"):
        y_pred = (y_pred_proba >= best_threshold).astype(int)
        f1_scr = f1_score(y_val, y_pred)
    else:
        f1_scr = np.nan

    return {
        "Model": model_name,
        "Mean PR-AUC": best_row["mean_test_pr_auc"],
        "Std PR-AUC": best_row["std_test_pr_auc"],
        "Mean ROC-AUC": best_row["mean_test_roc_auc"],
        "Std ROC-AUC": best_row["std_test_roc_auc"],
        "Best parameters": grid_search.best_params_,
        "Num of CV splits": grid_search.cv.n_splits,
        "Optimal threshold for F1-score": best_threshold,
        "Validation F1-score": f1_scr,
        "Validation PR-AUC": average_precision_score(y_val, y_pred_proba),
        "Validation ROC-AUC": roc_auc_score(y_val, y_pred_proba),
    }


summaries = []

### **3.3 Train LogisticRegression Model**

In [34]:
lr = LogisticRegression(
    l1_ratio=1,
    solver="saga",
    max_iter=5000,
    class_weight="balanced",
    random_state=42,
)
steps = [
    ("preproc", preproc_steps),
    ("scaler", StandardScaler()),
    ("classifier", lr)
]
pipeline = Pipeline(steps)
param_grid = {"classifier__C": [0.001, 0.01, 0.1, 1, 10]}
grid_search_lr = run_grid_search(X_train, y_train, pipeline, param_grid)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [35]:
best_threshold_lr, y_pred_proba_lr = select_threshold(grid_search_lr, X_val, y_val)
summaries.append(summarize_results("logistic", grid_search_lr, best_threshold_lr, y_val, y_pred_proba_lr))
pd.DataFrame(summaries)

,Model,Mean PR-AUC,Std PR-AUC,Mean ROC-AUC,Std ROC-AUC,Best parameters,Num of CV splits,Optimal threshold for F1-score,Validation F1-score,Validation PR-AUC,Validation ROC-AUC
0,logistic,0.186397,0.051218,0.726941,0.07157,{'classifier__C': 0.1},5,0.643078,0.204082,0.184371,0.667804


### **3.4 Train Ridge Model**

In [36]:
rc = RidgeClassifier(
    class_weight="balanced",
    random_state=42
)
steps = [
    ("preproc", preproc_steps),
    ("scaler", StandardScaler()),
    ("classifier", rc)
]
pipeline = Pipeline(steps)
param_grid = {"classifier__alpha": [0.001, 0.1, 1, 10]}
grid_search_rc = run_grid_search(X_train, y_train, pipeline, param_grid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [37]:
best_threshold_rc, y_pred_proba_rc = select_threshold(grid_search_rc, X_val, y_val)
summaries.append(summarize_results("ridge", grid_search_rc, best_threshold_rc, y_val, y_pred_proba_rc))
pd.DataFrame(summaries)

,Model,Mean PR-AUC,Std PR-AUC,Mean ROC-AUC,Std ROC-AUC,Best parameters,Num of CV splits,Optimal threshold for F1-score,Validation F1-score,Validation PR-AUC,Validation ROC-AUC
0,logistic,0.186397,0.051218,0.726941,0.07157,{'classifier__C': 0.1},5,0.643078,0.204082,0.184371,0.667804
1,ridge,0.175172,0.052247,0.684064,0.05616,{'classifier__alpha': 1},5,NaN,NaN,0.122781,0.679018


### **3.5 Train Decision Tree Model**

In [38]:
dt = DecisionTreeClassifier(
    class_weight="balanced",
    random_state=42
)
steps = [
    ("preproc", preproc_steps),
    ("classifier", dt)
]
pipeline = Pipeline(steps)
param_grid = {
    "classifier__max_depth": [13, 14, 15, 16, 17, 18],
    "classifier__min_samples_leaf": [6, 8, 10, 12, 14],
}
grid_search_dt = run_grid_search(X_train, y_train, pipeline, param_grid)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [39]:
best_threshold_dt, y_pred_proba_dt = select_threshold(grid_search_dt, X_val, y_val)
summaries.append(summarize_results("decision tree", grid_search_dt, best_threshold_dt, y_val, y_pred_proba_dt))
pd.DataFrame(summaries)

,Model,Mean PR-AUC,Std PR-AUC,Mean ROC-AUC,Std ROC-AUC,Best parameters,Num of CV splits,Optimal threshold for F1-score,Validation F1-score,Validation PR-AUC,Validation ROC-AUC
0,logistic,0.186397,0.051218,0.726941,0.071570,{'classifier__C': 0.1},5,0.643078,0.204082,0.184371,0.667804
1,ridge,0.175172,0.052247,0.684064,0.056160,{'classifier__alpha': 1},5,NaN,NaN,0.122781,0.679018
2,decision tree,0.157284,0.062354,0.591508,0.053116,"{'classifier__max_depth': 14, 'classifier__min...",5,0.000000,0.125373,0.068231,0.515602


### **3.6 Train Random Forest Model**

In [40]:
rf = RandomForestClassifier(
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)
steps = [
    ("preproc", preproc_steps),
    ("classifier", rf)
]
pipeline = Pipeline(steps)
param_grid = {
    "classifier__n_estimators": [300, 350, 400],
    "classifier__max_depth": [8, 9, 10, 11, 12],
    "classifier__min_samples_leaf": [7, 8, 9, 10],
    "classifier__max_features": [0.3, 0.5, 0.7],
}
grid_search_rf = run_grid_search(X_train, y_train, pipeline, param_grid)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [41]:
best_threshold_rf, y_pred_proba_rf = select_threshold(grid_search_rf, X_val, y_val)
summaries.append(summarize_results("random forest", grid_search_rf, best_threshold_rf, y_val, y_pred_proba_rf))
pd.DataFrame(summaries)

,Model,Mean PR-AUC,Std PR-AUC,Mean ROC-AUC,Std ROC-AUC,Best parameters,Num of CV splits,Optimal threshold for F1-score,Validation F1-score,Validation PR-AUC,Validation ROC-AUC
0,logistic,0.186397,0.051218,0.726941,0.071570,{'classifier__C': 0.1},5,0.643078,0.204082,0.184371,0.667804
1,ridge,0.175172,0.052247,0.684064,0.056160,{'classifier__alpha': 1},5,NaN,NaN,0.122781,0.679018
2,decision tree,0.157284,0.062354,0.591508,0.053116,"{'classifier__max_depth': 14, 'classifier__min...",5,0.000000,0.125373,0.068231,0.515602
3,random forest,0.233690,0.049820,0.757678,0.025587,"{'classifier__max_depth': 12, 'classifier__max...",5,0.189615,0.250000,0.176771,0.692995


### **3.7 Train XGBoost Model**

In [42]:
from xgboost import XGBClassifier
scale_pos_weight = sum(y == 0) / sum(y == 1)
bst = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    n_estimators=500,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42,
)
steps = [
    ("preproc", preproc_steps),
    ("classifier", bst)
]
pipeline = Pipeline(steps)
param_grid = {
    "classifier__max_depth": [3, 4, 5],
    "classifier__learning_rate": [0.0001, 0.005, 0.01],
    "classifier__subsample": [0.6, 0.7, 0.8],
    "classifier__colsample_bytree": [0.9, 1.0],
}
grid_search_bst = run_grid_search(X_train, y_train, pipeline, param_grid)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


C:\Users\dvfit\Code\semiconductor-manufacturing-line-testing\.venv\Lib\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [43]:
best_threshold_bst, y_pred_proba_bst = select_threshold(grid_search_bst, X_val, y_val)
summaries.append(summarize_results("XGBoost", grid_search_bst, best_threshold_bst, y_val, y_pred_proba_bst))
pd.DataFrame(summaries)

,Model,Mean PR-AUC,Std PR-AUC,Mean ROC-AUC,Std ROC-AUC,Best parameters,Num of CV splits,Optimal threshold for F1-score,Validation F1-score,Validation PR-AUC,Validation ROC-AUC
0,logistic,0.186397,0.051218,0.726941,0.071570,{'classifier__C': 0.1},5,0.643078,0.204082,0.184371,0.667804
1,ridge,0.175172,0.052247,0.684064,0.056160,{'classifier__alpha': 1},5,NaN,NaN,0.122781,0.679018
2,decision tree,0.157284,0.062354,0.591508,0.053116,"{'classifier__max_depth': 14, 'classifier__min...",5,0.000000,0.125373,0.068231,0.515602
3,random forest,0.233690,0.049820,0.757678,0.025587,"{'classifier__max_depth': 12, 'classifier__max...",5,0.189615,0.250000,0.176771,0.692995
4,XGBoost,0.246273,0.049305,0.759457,0.045980,"{'classifier__colsample_bytree': 1.0, 'classif...",5,0.299421,0.232558,0.138853,0.678043


### **3.8 Model Selection and Threshold Tuning**

The random forest model has the best scores based on the validation data. Therefore, this model is selected and retrained on the training and validation data.

In [44]:
best_pipeline = grid_search_rf.best_estimator_
best_pipeline.fit(X_full_train, y_full_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preproc', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('drop_nans', ...), ('drop_constant', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,nan_threshold,0.5
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`."

The threshold is tuned by computing the F1-score again, but this time based on out-of-fold (OOF) predictions made by fitting the model on the full_train dataset.

In [45]:
from sklearn.model_selection import cross_val_predict

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_proba = cross_val_predict(
    best_pipeline,
    X_full_train,
    y_full_train,
    cv=cv,
    method="predict_proba",
    n_jobs=-1
)[:, 1]

thresholds = np.linspace(0.01, 0.99, 99)
f1_scores = [
    f1_score(y_full_train, (oof_proba >= t).astype(int))
    for t in thresholds
]
best_threshold = thresholds[np.argmax(f1_scores)]
best_oof_f1 = np.max(f1_scores)
print(f"The best threshold value is {best_threshold:.2f} (F1-score of {best_oof_f1}).")

The best threshold value is 0.28 (F1-score of 0.26744186046511625).


### **3.9 Final Model Evaluation**

In [46]:
test_proba = best_pipeline.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= best_threshold).astype(int)

In [47]:
cv_rf_row = (
    pd.DataFrame(summaries)
    .query("Model == 'random forest'")
    .iloc[0]
)
cv_metrics = {
    "Dataset": "Training",
    "PR-AUC": cv_rf_row["Mean PR-AUC"],
    "ROC-AUC": cv_rf_row["Mean ROC-AUC"],
    "Decision threshold": np.nan,  # Not available from CV
    "F1-score": np.nan,  # Not available from CV
}
validation_metrics = {
    "Dataset": "Validation",
    "PR-AUC": cv_rf_row["Validation PR-AUC"],
    "ROC-AUC": cv_rf_row["Validation ROC-AUC"],
    "Decision threshold": cv_rf_row["Optimal threshold for F1-score"],
    "F1-score": cv_rf_row["Validation F1-score"],
}
oof_full_train_metrics = {
    "Dataset": "OOF full train", 
    "PR-AUC": average_precision_score(y_full_train, oof_proba),  
    "ROC-AUC": roc_auc_score(y_full_train, oof_proba),
    "Decision threshold": best_threshold,
    "F1-score": best_oof_f1,
}
test_metrics = {
    "Dataset": "Test",
    "PR-AUC": average_precision_score(y_test, test_proba),
    "ROC-AUC": roc_auc_score(y_test, test_proba),
    "Decision threshold": best_threshold,
    "F1-score": f1_score(y_test, test_pred),
}

final_results = pd.DataFrame([
    cv_metrics,
    validation_metrics,
    oof_full_train_metrics,
    test_metrics
])
final_results

,Dataset,PR-AUC,ROC-AUC,Decision threshold,F1-score
0,Training,0.233690,0.757678,NaN,NaN
1,Validation,0.176771,0.692995,0.189615,0.250000
2,OOF full train,0.173175,0.719184,0.280000,0.267442
3,Test,0.180779,0.783845,0.280000,0.171429


While the test PR-AUC is slightly better than the validation PR-AUC, the F1-score is lower. This could indicate some potential overfitting and need for further parameter tuning. Alternatively, the choice of F1-score as indicator for best threshold value could be revised. For the purpose of this project, the parameters obtained from the grid search will be used for the final model that will be deployed.

In [48]:
print("The following values are the best parameter values obtained for the random forest model")
grid_search_rf.best_params_

The following values are the best parameter values obtained for the random forest model


{'classifier__max_depth': 12,
 'classifier__max_features': 0.5,
 'classifier__min_samples_leaf': 8,
 'classifier__n_estimators': 350}